In [2]:
import pandas as pd
df_seoul = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df_seoul[:2]

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045


## 자연어 전처리

In [3]:
stopwords = []

In [8]:
# 토크나이징 함수 정의
from konlpy.tag import Okt
okt = Okt()

def tokenizer(raw, pos=["Noun","Alpha","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [9]:
# tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer)

In [70]:
features = tfidfVectorizer.fit_transform(df_seoul['내용'])
features.toarray()[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 유사도확인

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
def similarity(x):
    result_list = cosine_similarity(features.toarray()[x:x+1], features.toarray())
    df_seoul['유사도'] = result_list[0]
    return df_seoul.sort_values(by='유사도', ascending=False)

In [82]:
df_seoul.query('번호==1772').index[0]

873

In [84]:

similarity(1772).head(6) # 3번에 대한 유사도 검증

,번호,분류,제목,내용,내용번호,유사도
1772,873,경제,도시계획시설부지 재결신청 이후 진행단계는 어떤 과정을 거칩니까?,도시계획시설부지 재결신청 이후 진행단계는 어떤 과정을 거칩니까재결신청 이후 재결신청...,2897109,1.000000
1775,870,경제,도시계획시설 재결신청은 어떻게 합니까?,도시계획시설 재결신청은 어떻게 합니까재결신청은 소유자 및 사업시행자가 모두 할 수 ...,2897108,0.779276
1771,874,경제,도시계획시설부지의 재결과정에서는 의견을 제출할 수 없는지요?,도시계획시설부지의 재결과정에서는 의견을 제출할 수 없는지요재결신청 이후 신청서 및 ...,2897110,0.624567
1780,865,경제,도시계획시설에 편입된 보상물건의 재결신청 청구는 몇일 이내에 해야 하나요?,도시계획시설에 편입된 보상물건의 재결신청 청구는 몇일 이내에 해야 하나요사업인정 고...,2897106,0.584405
1768,877,경제,도시계획시설부지에 포함된 건물보상금액의 불만입니다.,도시계획시설부지에 포함된 건물보상금액의 불만입니다사업시행자로 하여금 재결신청을 청구...,2897114,0.558815
1767,878,경제,도시계획시설 부지의 이의재결이란 무엇입니까?,도시계획시설 부지의 이의재결이란 무엇입니까수용재결에 대한 이의가 있는 경우 중앙토지...,2897115,0.510080
